<a href="https://colab.research.google.com/github/takatakamanbou/Vision/blob/main/Vision2021_ex05b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision2021-ex05b

課題の期限や提出の方法などについては，Visionチーム内に書いてます．

# MNISTのロジスティック回帰



## 準備

In [ ]:
import numpy as np
import scipy as sp
#from scipy.misc import logsumexp     # for older SciPy 
from scipy.special import logsumexp     # for scipy.misc.logsumexp
import matplotlib.pyplot as plt

## MNIST の入手

MNIST についてはこちらを参考: http://yann.lecun.com/exdb/mnist/

In [ ]:
# 上記サイトから 4 つのファイルを入手し， gunzip
! for fn in train-images-idx3-ubyte t10k-images-idx3-ubyte train-labels-idx1-ubyte t10k-labels-idx1-ubyte; do if [ ! -e ${fn} ]; then wget -nc http://yann.lecun.com/exdb/mnist/${fn}.gz ; gunzip ${fn}.gz; fi; done
! ls -l

## MNIST を扱う関数の定義と動作確認

In [ ]:
# MNIST のためのクラスの定義

import struct
import os
import numpy as np

class MNIST:
    
    def __init__(self):

        fnImageL = 'train-images-idx3-ubyte'
        fnImageT = 't10k-images-idx3-ubyte'
        fnLabelL = 'train-labels-idx1-ubyte'
        fnLabelT = 't10k-labels-idx1-ubyte'
    
        if not os.path.exists(fnImageL) or not os.path.exists(fnImageT) or not os.path.exists(fnLabelL) or not os.path.exists(fnLabelT):
            print('Please get the MNIST files first.')
            return
       
        self.fnImage = {'L': fnImageL, 'T': fnImageT}
        self.fnLabel  = {'L': fnLabelL, 'T': fnLabelT}
        self.nrow = 28
        self.ncol = 28
        self.nclass = 10
        
        
    def getLabel( self, LT ):
        
        return _readLabel( self.fnLabel[LT] )
 
 
    def getImage( self, LT ):
        
        return _readImage( self.fnImage[LT] )
 
 
##### reading the label file
#
def _readLabel( fnLabel ):
 
    f = open( fnLabel, 'rb' )
 
    ### header (two 4B integers, magic number(2049) & number of items)
    #
    header = f.read( 8 )
    mn, num = struct.unpack( '>2i', header )  # MSB first (bigendian)
    assert mn == 2049
    #print mn, num
 
    ### labels (unsigned byte)
    #
    label = np.array( struct.unpack( '>%dB' % num, f.read() ), dtype = int )
 
    f.close()
 
    return label

 
##### reading the image file
#
def _readImage( fnImage ):
 
    f = open( fnImage, 'rb' )
 
    ### header (four 4B integers, magic number(2051), #images, #rows, and #cols
    #
    header = f.read( 16 )
    mn, num, nrow, ncol = struct.unpack( '>4i', header ) # MSB first (bigendian)
    assert mn == 2051
    #print mn, num, nrow, ncol
 
    ### pixels (unsigned byte)
    #
    npixel = ncol * nrow
    #pixel = np.empty( ( num, npixel ), dtype = int )
    #pixel = np.empty( ( num, npixel ), dtype = np.int32 )
    pixel = np.empty( ( num, npixel ) )
    for i in range( num ):
        buf = struct.unpack( '>%dB' % npixel, f.read( npixel ) )
        pixel[i, :] = np.asarray( buf )
 
    f.close()
 
    return pixel
        

In [ ]:
### MNIST クラスの動作確認 ###

mnist = MNIST()
print(mnist.nrow, mnist.ncol, mnist.nclass)

print( '# MNIST training data' )
dat = mnist.getImage( 'L' )
lab = mnist.getLabel( 'L' )
print( dat.shape, dat.dtype, lab.shape, lab.dtype )
 
print( '# MNIST test data' )
dat = mnist.getImage( 'T' )
lab = mnist.getLabel( 'T' )
print( dat.shape, dat.dtype, lab.shape, lab.dtype )    

In [ ]:
### MNIST データを可視化するプログラム ###

import cv2
import numpy as np
import matplotlib.pyplot as plt

nx, ny = 16, 8  #  横縦に並べる画像の数
gap = 4         #  画像間のスペース

# MNIST の学習データ 
mn = MNIST()
dat = mn.getImage('L')
lab = mn.getLabel('L')
datS = dat[:nx*ny]  # 最初の nx * ny 個だけ取り出す
labS = lab[:nx*ny]  # 最初の nx * ny 個だけ取り出す
nrow, ncol = mn.nrow, mn.ncol

# 並べた画像の幅と高さ
width  = nx * (ncol + gap) + gap
height = ny * (nrow + gap) + gap

# 画像の作成
img = np.zeros((height, width), dtype = int) + 128
for iy in range(ny):
    lty = iy*(nrow + gap) + gap
    for ix in range(nx):
        ltx = ix*(ncol + gap) + gap
        img[lty:lty+nrow, ltx:ltx+ncol] = datS[iy*nx+ix].reshape((nrow, ncol))

# 画像の出力
plt.axis('off')
plt.imshow(img, cmap = 'gray')
plt.show()

# ラベルの出力
print(labS.reshape((ny, nx)))

## 学習のための関数の定義

In [ ]:
### ロジスティック回帰モデル
#
def  model(X, W, b):
    
    # X, W, b が正しい shape かチェック
    assert X.ndim == 2   # (batchsize, D)
    assert W.ndim == 2  # (K, D) 
    assert b.ndim == 1   # (K,)
    K, D = W.shape
    assert X.shape[1] == D
    assert b.shape[0] == K
    
    # softmax の手前の値
    Y = X @ W.T + b[np.newaxis, :]  # (batchsize, K)
    
    # softmax （普通の計算法）
    #Ztmp = np.exp(Y)  # (batchsize, K)
    #Z = Ztmp / np.sum(Ztmp, axis = 1, keepdims = True)  # (batchsize, K)
    
    # softmax （logsumexp経由，こうする理由は後述）
    lse = logsumexp(Y, axis = 1, keepdims = True)  # (batchsize,1)
    Z = np.exp(Y - lse)                                                  # (batchsize, K)
    
    return Y, Z

In [ ]:
### 交差エントロピー （logsumexp version）
#
def cross_entropy2(Y, lab):
    
    # shape の整合性をチェック
    assert Y.ndim == 2      # (n, K)
    assert lab.ndim == 1   # (n,)
    n, K = Y.shape
    assert Y.shape[0] == n

    # 交差エントロピー
    Yk = np.choose(lab, Y.T)                 # Y の各行について， lab の値に該当する列の値のみ取り出す
    H = - Yk + logsumexp(Y, axis = 1)  # こうやって計算する理由は後述
        
    return H

In [ ]:
### パラメータの更新
#
def update(X, Z, Zt, W, b, eta):
    
    Zt_Z = Zt - Z
    
    # update W
    #dHdW = ##### ここを書きましょう #####
    W += eta * dHdW
    
    # update b
    #dHdb  = ##### ここを書きましょう #####
    b += eta * dHdb

## データの用意

In [ ]:
# 学習データと検査データ

mn = MNIST()
D = mn.nrow * mn.ncol
K = mn.nclass

datLraw = mn.getImage('L') / 255.0  #  [0, 255] => [0,1]
labLraw = mn.getLabel('L')
datL = datLraw[:50000, :]   # 6万個のうち最初の5万個を学習用に
labL = labLraw[:50000]
datV = datLraw[50000:, :]   # 残りは検査(validation)用に
labV = labLraw[50000:]

nL = datL.shape[0]
nV = datV.shape[0]

In [ ]:
# テストデータ

datT = mn.getImage('T') / 255.0  #  [0, 255] => [0,1]
labT = mn.getLabel('T')
nT = datT.shape[0]

## 学習させる

In [ ]:
# 教師信号の生成
ZtL = np.zeros((nL, K))
for k in range(K):
    ZtL[labL == k, k] = 1.0

# パラメータの初期化
W = 0.01 * np.random.random((K, D))
b = 0.01 * np.random.random(K)

# 実験条件の設定
nitr = 1000
eta = 0.01
batchsize = 64

# 学習の繰り返し
for i in range(nitr):
    
    # バッチの作成（ここでは毎回ランダムに選択）
    idx = np.random.randint(nL, size = batchsize)
    X = datL[idx, :]
    Zt = ZtL[idx, :]
    
    # 出力の計算とパタメータの更新
    Y, Z = model(X, W, b)
    update(X, Z, Zt, W, b, eta)
    
    # 適当なタイミングで交差エントロピーと識別率を表示
    if i < 10 or i % 50 == 0:
        # 学習データの交差エントロピーと正答数
        Y, Z = model(datL, W, b)
        HL = cross_entropy2(Y, labL)
        correctL = np.sum(labL == np.argmax(Z, axis = 1)) 
        # 検査データの交差エントロピーと正答数
        Y, Z = model(datV, W, b)
        HV = cross_entropy2(Y, labV)
        correctV = np.sum(labV == np.argmax(Z, axis = 1)) 

        print('{0}   {1:.5f} {2:.3f}     {3:.5f} {4:.3f}'.format(i, np.mean(HL), correctL / nL, np.mean(HV), correctV / nV))

## テストする



In [ ]:
i = nitr

Y, Z = model(datL, W, b)
HL = cross_entropy2(Z, labL)
correctL = np.sum(labL == np.argmax(Z, axis = 1))

Y, Z = model(datV, W, b)
HV = cross_entropy2(Z, labV)
correctV = np.sum(labV == np.argmax(Z, axis = 1))

Y, Z = model(datT, W, b)
HT = cross_entropy2(Z, labT)
correctT = np.sum(labT == np.argmax(Z, axis = 1))

print('{0}   {1:.5f} {2:.3f}     {3:.5f} {4:.3f}     {5:.5f} {6:.3f}'.format(i, np.mean(HL), correctL / nL, np.mean(HV), correctV / nV, np.mean(HT), correctT / nT))

# MNIST の最近傍法

## かしこいやり方

$||\mathbf{x}||^2$ を求めておく

In [ ]:
X2 = np.sum(datL**2, axis = 1)

テストデータと学習データたちとの内積を使って最近傍法

In [ ]:
%%time
# ↑このセルの実行時間を計測

N = 1000 # テストデータほんとは1万個あるけど最初のN個で勘弁してやる
out = np.empty(N, dtype=int)  # 識別結果を入れる

for n in range(N):
    if n % 100 == 0:
        print(n)
    d2 = X2 - 2 * (datL @ datT[n]) # ここ括弧重要 https://twitter.com/takataka8191/status/1132542307427708928?s=20
    out[n] = labL[np.argmin(d2)]


cc = np.sum(out == labT[:N])  # 正解数
print('{0}/{1} = {2:.2f}%'.format(cc, N, cc/N*100))

## ふつーのやり方

In [ ]:
%%time
# ↑このセルの実行時間を計測

N = 1000 # テストデータほんとは1万個あるけど最初のN個で勘弁してやる
out = np.empty(N, dtype=int)  # 識別結果を入れる

for n in range(N):  
    if n % 100 == 0:
        print(n)
    d2 = np.sum(np.square(datL - datT[n, np.newaxis, :]), axis = 1)  # 普通にユークリッド距離を計算
    out[n] = labL[np.argmin(d2)]


cc = np.sum(out == labT[:N])  # 正解数
print('{0}/{1} = {2:.2f}%'.format(cc, N, cc/N*100))